# Naver 영화 한줄평 crawling & Text Preprocessing

In [6]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

##  한줄평 crawling

In [48]:
url = 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=187321'
resp = requests.get( url )
html = BeautifulSoup( resp.content, 'lxml' )
score_result = html.find( 'div', { 'class' : 'score_result' } )
lis = score_result.findAll( 'li' )

comments = []
for li in lis:
    review_text = li.find( 'p' ).getText()
    comments.append( review_text.strip() )

for comment in comments:
    print( comment + '\n' )

이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.

충무로: 이거 어케하는거냐?

아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.

촬영감독의 영혼까지 갈아넣은 마스터피스

오스카 작품상 탔어도 할말 없었을것 같다.



In [49]:
with open( 'comment_1917.txt', 'w', encoding = 'utf-8' ) as f:
    for comment in comments:
        f.write( comment + '\n' )

movie_comments = []
with open( 'comment_1917.txt', 'r', encoding = 'utf-8' ) as f:
    lines = f.readlines()
    for line in lines:
        movie_comments.append( line.rstrip( '\n' ) )

print( movie_comments )

['이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.', '충무로: 이거 어케하는거냐?', '아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.', '촬영감독의 영혼까지 갈아넣은 마스터피스', '오스카 작품상 탔어도 할말 없었을것 같다.']


In [61]:
movie_comment = ''
for comment in movie_comments:
    movie_comment += comment
print( movie_comment )

이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.충무로: 이거 어케하는거냐?아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.촬영감독의 영혼까지 갈아넣은 마스터피스오스카 작품상 탔어도 할말 없었을것 같다.


## 형태소 분석

In [68]:
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt

In [52]:
okt = Okt()

###  형태소 추출

In [62]:
morphemes = []
for morphs in okt.morphs( movie_comment ):
    morphemes.append( morphs )
print( morphemes )

['이', '영화', '는', '미쳤다', '.', '넷플릭스', '가', '일상', '화', '된', '시대', '에', '극장', '이', '존재', '해야하는', '이유', '를', '증명', '해준다', '.', '충무로', ':', '이', '거', '어케', '하는거냐', '?', '아카데미', '에서', '촬영상', ',', '음향효과', '상', ',', '시', '각', '효과', '상', '을', '받은', '이유', '가', '고스', '란', '히', '녹아있는', '영화', '.', 'IMAX', '로', '관람', '하', '는걸', '추천', '한다', '.', '촬영감독', '의', '영혼', '까지', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것', '같다', '.']


###  형태소 추출 및 어간 추출

In [63]:
morphemes = []
for morphs in okt.morphs( movie_comment, stem = True ):
    morphemes.append( morphs )
print( morphemes )

['이', '영화', '는', '미치다', '.', '넷플릭스', '가', '일상', '화', '되다', '시대', '에', '극장', '이', '존재', '하다', '이유', '를', '증명', '해주다', '.', '충무로', ':', '이', '거', '어케', '하다', '?', '아카데미', '에서', '촬영상', ',', '음향효과', '상', ',', '시', '각', '효과', '상', '을', '받다', '이유', '가', '고스', '란', '히', '녹다', '영화', '.', 'IMAX', '로', '관람', '하', '는걸', '추천', '하다', '.', '촬영감독', '의', '영혼', '까지', '갈아', '넣다', '마스터피스', '오스카', '작품', '상', '타다', '하다', '없다', '같다', '.']


### 형태소 추출 및 정규화

In [64]:
morphemes = []
for morphs in okt.morphs( movie_comment, norm = True ):
    morphemes.append( morphs )
print( morphemes )

['이', '영화', '는', '미쳤다', '.', '넷플릭스', '가', '일상', '화', '된', '시대', '에', '극장', '이', '존재', '해야하는', '이유', '를', '증명', '해준다', '.', '충무로', ':', '이', '거', '어떻게', '하는거냐', '?', '아카데미', '에서', '촬영상', ',', '음향효과', '상', ',', '시', '각', '효과', '상', '을', '받은', '이유', '가', '고스', '란', '히', '녹아있는', '영화', '.', 'IMAX', '로', '관람', '하', '는걸', '추천', '한다', '.', '촬영감독', '의', '영혼', '까지', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것', '같다', '.']


### 명사 분리

In [65]:
morphemes = []
for morphs in okt.nouns( movie_comment ):
    morphemes.append( morphs )
print( morphemes )

['이', '영화', '넷플릭스', '일상', '시대', '극장', '존재', '이유', '증명', '충무로', '거', '어케', '아카데미', '촬영상', '음향효과', '효과', '이유', '고스', '란', '영화', '로', '관람', '는걸', '추천', '촬영감독', '영혼', '마스터피스', '오스카', '작품']


### 어절 분리

In [66]:
morphemes = []
for morphs in okt.phrases( movie_comment ):
    morphemes.append( morphs )
print( morphemes )

['이 영화', '넷플릭스', '일상화', '일상화된 시대', '극장', '존재', '존재해야하는 이유', '증명', '충무로', '이거', '이거 어케', '아카데미', '촬영상', '음향효과상', '시각효과상', '이유', '고스란', '영화', 'IMAX로', 'IMAX로 관람하는걸', 'IMAX로 관람하는걸 추천', '촬영감독', '촬영감독의 영혼', '마스터피스오스카', '마스터피스오스카 작품상', '시대', '어케', '고스', 'IMAX', '관람하', '는걸', '추천', '영혼', '마스터피스', '오스카', '작품상']


### 품사 태깅

In [67]:
morphemes = []
for morphs in okt.pos( movie_comment, join = True ):
    morphemes.append( morphs )
print( morphemes )

['이/Noun', '영화/Noun', '는/Josa', '미쳤다/Adjective', './Punctuation', '넷플릭스/Noun', '가/Josa', '일상/Noun', '화/Suffix', '된/Verb', '시대/Noun', '에/Josa', '극장/Noun', '이/Josa', '존재/Noun', '해야하는/Verb', '이유/Noun', '를/Josa', '증명/Noun', '해준다/Verb', './Punctuation', '충무로/Noun', ':/Punctuation', '이/Determiner', '거/Noun', '어케/Noun', '하는거냐/Verb', '?/Punctuation', '아카데미/Noun', '에서/Josa', '촬영상/Noun', ',/Punctuation', '음향효과/Noun', '상/Suffix', ',/Punctuation', '시/Modifier', '각/Modifier', '효과/Noun', '상/Suffix', '을/Josa', '받은/Verb', '이유/Noun', '가/Josa', '고스/Noun', '란/Noun', '히/Adverb', '녹아있는/Verb', '영화/Noun', './Punctuation', 'IMAX/Alpha', '로/Noun', '관람/Noun', '하/Suffix', '는걸/Noun', '추천/Noun', '한다/Verb', './Punctuation', '촬영감독/Noun', '의/Josa', '영혼/Noun', '까지/Josa', '갈아/Adverb', '넣은/Verb', '마스터피스/Noun', '오스카/Noun', '작품/Noun', '상/Suffix', '탔어도/Verb', '할말/Verb', '없었을것/Adjective', '같다/Adjective', './Punctuation']


### 불용어 제거

In [69]:
stop_words = []
with open( 'stopword.txt', 'r', encoding = 'utf-8' ) as f:
    lines = f.readlines()
    for line in lines:
        stop_words.append( line.rstrip( '\n' ) )

print( stop_words )

FileNotFoundError: [Errno 2] No such file or directory: 'stopword.txt'

In [ ]:
result = [
for word in okt.morphs( movie_comment ):
    morphemes.append( morphs )
print( morphemes )